In [1]:
import numpy as np
import random
from concurrent import futures
from functools import partial

In [2]:
class FlipSimulation:
    def __init__(self):
        self.stateVal = dict([('HHH', 7), ('HHT', 6), ('HTH', 5), ('HTT', 4), ('THH', 3), ('THT', 2), ('TTH', 1), ('TTT', 0)])

    def setSeed(self, seed):
        np.random.seed(seed)

    def simulate(self, x:str, y:str, times=2000):
        res = [0, 0]
        sim = partial(self._sim, x=self.stateVal[x], y=self.stateVal[y])
        for i in range(times):
            res[sim()] += 1
        return res

    def _sim(self, x:int, y:int):
        mem = np.array([-10, 2 if np.random.rand()>0.5 else 0, 1 if np.random.rand() > 0.5 else 0])
        while True:
            mem[0] = 2*mem[1]
            mem[1] = 2*mem[2]
            mem[2] = 1 if np.random.rand() > 0.5 else 0
            num = np.sum(mem)
            assert(mem[0] == 0 or mem[0] == 4)
            assert(mem[1] == 2 or mem[1] == 0)
            if num == x and num != y:
                return 0
            elif num != x and num == y:
                return 1
model = FlipSimulation()

In [3]:
def cal(x, y):
    res = model.simulate(x,y)
    print(x, y, res, res[0]/res[1], res[1]/res[0])

In [4]:
s = ['TTT', 'TTH', 'THT', 'THH', 'HTT', 'HTH', 'HHT', 'HHH']
cal(s[7], s[3])
cal(s[6], s[5])
cal(s[6], s[4])
cal(s[6], s[3])
cal(s[4], s[0])
cal(s[4], s[1])
cal(s[1], s[2])
cal(s[1], s[3])

HHH THH [248, 1752] 0.1415525114155251 7.064516129032258
HHT HTH [1337, 663] 2.0165912518853695 0.49588631264023936
HHT HTT [1312, 688] 1.9069767441860466 0.524390243902439
HHT THH [519, 1481] 0.350438892640108 2.8535645472061657
HTT TTT [1727, 273] 6.326007326007326 0.1580775911986103
HTT TTH [1487, 513] 2.898635477582846 0.3449899125756557
TTH THT [1345, 655] 2.053435114503817 0.48698884758364314
TTH THH [1336, 664] 2.0120481927710845 0.49700598802395207


In [5]:
x = [[0, 0], [1, 1]]
y = [0.4, 0.6]
np.einsum('ij,j->i', x, y)

array([0., 1.])

In [6]:
def RandomWalk(edges, d, epsilon):
    assert (1 >= d >= 0)
    M = np.zeros([len(edges), len(edges)])
    plus = np.ones(len(edges)) * (1-d)
    R = np.ones(len(edges))
    for i in range(len(edges)):
        num = len(edges[i])
        if num == 0:
            M[i][i] = 0#1
        else:
            p = 1/num
            for v in edges[i]:
                M[v][i] = p
    dM = d*M
    print(M)
    delta = 2 + epsilon
    counter = 0
    while delta > epsilon:
        tmp = np.einsum('ij,j->i', dM, R) + plus
        delta = np.sum(np.abs(tmp - R))
        R = tmp
        counter += 1
    print(counter)
    return R, M, np.einsum('ij,j->i', dM, R) + plus

In [7]:
vertex = [[], [2, 4], [0], [1, 4], [3], [2]]
RandomWalk(vertex, 0.85, 1e-20)

[[0.  0.  1.  0.  0.  0. ]
 [0.  0.  0.  0.5 0.  0. ]
 [0.  0.5 0.  0.  0.  1. ]
 [0.  0.  0.  0.  1.  0. ]
 [0.  0.5 0.  0.5 0.  0. ]
 [0.  0.  0.  0.  0.  0. ]]
128


(array([0.54501392, 0.44052296, 0.46472226, 0.68358344, 0.62774522,
        0.15      ]),
 array([[0. , 0. , 1. , 0. , 0. , 0. ],
        [0. , 0. , 0. , 0.5, 0. , 0. ],
        [0. , 0.5, 0. , 0. , 0. , 1. ],
        [0. , 0. , 0. , 0. , 1. , 0. ],
        [0. , 0.5, 0. , 0.5, 0. , 0. ],
        [0. , 0. , 0. , 0. , 0. , 0. ]]),
 array([0.54501392, 0.44052296, 0.46472226, 0.68358344, 0.62774522,
        0.15      ]))

In [8]:
vertex = [[],[0], [0], [1], [1], [2], [2], [2]]
RandomWalk(vertex, 0.85, 1e-20)

[[0. 1. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 1. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]]
4


(array([0.946875, 0.405   , 0.5325  , 0.15    , 0.15    , 0.15    ,
        0.15    , 0.15    ]),
 array([[0., 1., 1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 1., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1., 1., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.]]),
 array([0.946875, 0.405   , 0.5325  , 0.15    , 0.15    , 0.15    ,
        0.15    , 0.15    ]))

In [ ]:
vertex =
a, b, c = RandomWalk(vertex, , 1e-20)
a

[[0.         0.         0.         0.         0.         0.
  0.33333333 0.        ]
 [0.5        0.         0.5        0.33333333 0.         0.
  0.         0.        ]
 [0.5        0.         0.         0.         0.         0.
  0.         0.        ]
 [0.         1.         0.         0.         0.         0.
  0.         0.        ]
 [0.         0.         0.5        0.33333333 0.         0.
  0.33333333 0.        ]
 [0.         0.         0.         0.33333333 0.33333333 0.
  0.         0.5       ]
 [0.         0.         0.         0.         0.33333333 0.
  0.         0.5       ]
 [0.         0.         0.         0.         0.33333333 1.
  0.33333333 0.        ]]


In [ ]:
print(a)